In [7]:
import galois
from collections import Counter
import itertools

# Define GF(2)
GF2 = galois.GF(2)

# Define function F

def F_0(x0, x1, x2):
    return x0 * x0 + x1 * x2 + x2 * x1  # Placeholder function

def F_1(x0, x1, x2):
    return x0 + x1 + x2  # Placeholder function

def F_2(x0, x1, x2):
    return x0 * x2 + x1  # Placeholder function

def generate_GL3_F2():
    """Generates all invertible linear transformations L(x0, x1, x2) with det != 0."""
    GL3_F2 = []
    elements = [0, 1]  # Use standard Python integers for determinant calculation
    for a0, b0, c0, a1, b1, c1, a2, b2, c2 in itertools.product(elements, repeat=9):
        det = (a0 * (b1 * c2 - b2 * c1) - b0 * (a1 * c2 - a2 * c1) + c0 * (a1 * b2 - a2 * b1)) % 2
        if det == 1:  # Ensure determinant is 1
            GL3_F2.append((GF2(a0), GF2(b0), GF2(c0), GF2(a1), GF2(b1), GF2(c1), GF2(a2), GF2(b2), GF2(c2)))
    return GL3_F2

# Generate GL3(F2)
GL3_F2 = generate_GL3_F2()

def compute_L_derivative(xa, x, L, func):
    """Computes the L-derivative: LDaF(x) = F(x + a) - L(F(x))"""
    x_a0, x_a1, x_a2 = xa
    x0, x1, x2 = x
    a0, b0, c0, a1, b1, c1, a2, b2, c2 = L
    
    # Compute F(x + a)
    Fx_a0 = func[0](x_a0, x_a1, x_a2)
    Fx_a1 = func[1](x_a0, x_a1, x_a2)
    Fx_a2 = func[2](x_a0, x_a1, x_a2)
    
    # Compute L(F(x))
    Fx0 = func[0](x0, x1, x2)
    Fx1 = func[1](x0, x1, x2)
    Fx2 = func[2](x0, x1, x2)
    LFx0 = a0 * Fx0 + b0 * Fx1 + c0 * Fx2
    LFx1 = a1 * Fx0 + b1 * Fx1 + c1 * Fx2
    LFx2 = a2 * Fx0 + b2 * Fx1 + c2 * Fx2
    
    # Compute LDaF(x) = F(x + a) - L(F(x))
    LD_F0 = Fx_a0 - LFx0
    LD_F1 = Fx_a1 - LFx1
    LD_F2 = Fx_a2 - LFx2
    
    return (int(LD_F0), int(LD_F1), int(LD_F2))

# Generate all possible values for a, x
a_values = list(itertools.product([GF2(0), GF2(1)], repeat=3))
x_values = list(itertools.product([GF2(0), GF2(1)], repeat=3))

# Compute LDaF for all a, x, and L ∈ GL3(F2)
results = []
total_counter = Counter()
a_counters = {tuple(map(int, a)): Counter() for a in a_values}

for a in a_values:
    a_tuple = tuple(map(int, a))
    for x in x_values:
        x_tuple = tuple(map(int, x))
        x_a = tuple(x[i] + a[i] for i in range(3))  # Compute x + a using GF(2) addition
        for L in GL3_F2:
            L_tuple = f"L({', '.join(map(str, L))})"
            for func in [(F_0, F_1, F_2)]:
                LD_F = compute_L_derivative(x_a, x, L, func)
                results.append((L_tuple, a_tuple, x_tuple, LD_F))
                a_counters[a_tuple][tuple(LD_F)] += 1  # Counter for each a
                total_counter[tuple(LD_F)] += 1  # Total counter

# Sort results by L -> a -> x
results.sort()

# Display results
print("\n## Final Results (binary):")
previous_a = None
for L_tuple, a_tuple, x_tuple, (expr1, expr2, expr3) in results:
    if previous_a is not None and previous_a != a_tuple:
        print()  # Add space between different a values
    print(f"L = {L_tuple}, a = {a_tuple}, x = {x_tuple}: (LD_F0 = {expr1}, LD_F1 = {expr2}, LD_F2 = {expr3})")
    previous_a = a_tuple

# Display count of different results per a
print("\n## Result Counts Per a:")
for a, counter in sorted(a_counters.items()):
    print(f"For a = {a}:")
    for key, count in sorted(counter.items()):
        print(f"  Result {key}: {count} occurrences")
    print()

# Display total count of different results
print("\n## Total Result Counts:")
for key, count in sorted(total_counter.items()):
    print(f"Result {key}: {count} occurrences")



## Final Results (binary):
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (0, 0, 0): (LD_F0 = 0, LD_F1 = 0, LD_F2 = 0)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (0, 0, 1): (LD_F0 = 0, LD_F1 = 0, LD_F2 = 0)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (0, 1, 0): (LD_F0 = 1, LD_F1 = 0, LD_F2 = 1)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (0, 1, 1): (LD_F0 = 1, LD_F1 = 0, LD_F2 = 1)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (1, 0, 0): (LD_F0 = 1, LD_F1 = 0, LD_F2 = 1)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (1, 0, 1): (LD_F0 = 0, LD_F1 = 0, LD_F2 = 0)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (1, 1, 0): (LD_F0 = 0, LD_F1 = 0, LD_F2 = 0)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 0), x = (1, 1, 1): (LD_F0 = 1, LD_F1 = 0, LD_F2 = 1)

L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 1), x = (0, 0, 0): (LD_F0 = 0, LD_F1 = 1, LD_F2 = 0)
L = L(0, 0, 1, 0, 1, 0, 1, 0, 0), a = (0, 0, 1), x = (0, 0, 1): (LD_F0 = 0, LD_F1 = 1, LD